# scGPT: Preprocessing Code

scGPT performs preprocessing within its embedding function from a counts matrix

In [4]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import mode
import scanpy as sc
import sklearn
import warnings

In [5]:
import scgpt as scg
import faiss

/home/jpic/.conda/envs/scgpt3/lib/python3.10/site-packages/scgpt/model/model.py:21: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
/home/jpic/.conda/envs/scgpt3/lib/python3.10/site-packages/scgpt/model/multiomic_model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")


## Helper Methods

## Set Data & Parameters

In [20]:
input_file = '/nfs/turbo/umms-indikar/shared/projects/geneformer/data/rajapakse_lab_data_jpic.h5ad'
model_dir     = Path("/nfs/turbo/umms-indikar/shared/projects/foundation_models/scGPT_human")
adata         = sc.read_h5ad(input_file)
cell_type_key = list(adata.obs.columns) #"dataset"
gene_col      = "index"
output_directory = '/nfs/turbo/umms-indikar/shared/projects/foundation_models/example_inputs/tests'
verbose       = True

In [16]:
list(adata.obs.columns)

['dataset',
 'sample_id',
 'timepoint',
 'hour',
 'n_counts',
 'control',
 'order',
 'replicate',
 'batch']

In [41]:
output_directory = '~/.cache/embeddings/'
full_path = os.path.expanduser(output_directory)
# Create the directory if it doesn't exist
os.makedirs(full_path, exist_ok=True)


## Preprocess Data

In [21]:
ref_embed_adata = scg.tasks.embed_data(
    adata,
    model_dir,
    gene_col=gene_col,
    obs_to_save=cell_type_key,  # optional arg, only for saving metainfo
    batch_size=64,
    return_new_adata=True,
)

scGPT - INFO - match 19267/19393 genes in vocabulary of size 60697.


/home/jpic/.conda/envs/scgpt3/lib/python3.10/site-packages/scgpt/model/model.py:77: UserWarning: flash-attn is not installed, using pytorch transformer instead. Set use_fast_transformer=False to avoid this warning. Installing flash-attn is highly recommended.
  warnings.warn(
Embedding cells: 100%|██████████| 2/2 [00:01<00:00,  1.82it/s]
/home/jpic/.conda/envs/scgpt3/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


In [22]:
ref_embed_adata.obs

,dataset,sample_id,timepoint,hour,n_counts,control,order,replicate,batch
data_id,,,,,,,,,
S1a,chen_2015,S1a,0.0,0.0,7901832,True,1,1,0
S1b,chen_2015,S1b,0.0,0.0,8113329,True,1,2,0
S2a,chen_2015,S2a,0.0,0.0,9831046,False,2,1,0
S2b,chen_2015,S2b,0.0,0.0,10123271,False,2,2,0
S3a,chen_2015,S3a,1.0,8.0,10490839,False,3,1,0
...,...,...,...,...,...,...,...,...,...
63275_T11R3,liu_2018,63275,3.0,80.0,13515971,False,11,3,1
63290_T12R3,liu_2018,63290,1.0,88.0,9522866,False,12,3,1
63287_T13R3,liu_2018,63287,2.0,96.0,12370157,False,13,3,1


In [23]:
sc.pp.neighbors(ref_embed_adata, use_rep="X")
sc.tl.umap(ref_embed_adata)
sc.pl.umap(ref_embed_adata, color=cell_type_key, frameon=False, wspace=0.4)

AttributeError: module 'numba' has no attribute 'core'